# EODAG as STAC client

## STAC API
EODAG can perform an item search over a STAC compliant API. Found STAC items are returned as `EOProduct` objects with STAC metadata mapped to OGC OpenSearch Extension for Earth Observation.

EODAG comes with already configured providers, but you can also add new ones dynamically.

In [1]:
import os
# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
from eodag.utils.logging import setup_logging
setup_logging(verbose=1)

from eodag.api.core import EODataAccessGateway

dag = EODataAccessGateway()

2021-03-05 14:27:51,861-15s eodag.config                     [INFO    ] Loading user configuration from: /home/sylvain/.config/eodag/eodag.yml
2021-03-05 14:27:52,069-15s eodag.core                       [INFO    ] Locations configuration loaded from /home/sylvain/.config/eodag/locations.yml


List already configured providers providing a STAC API compliant service:

In [2]:
[p.name for p in dag.providers_config.values() if hasattr(p, "search") and p.search.type == 'StacSearch']

['astraea_eod', 'usgs_satapi_aws']

Then, let's update EODAG's configuration with a new STAC provider

In [3]:
dag.update_providers_config("""
    tamn:
        search:
            type: StacSearch
            api_endpoint: https://tamn.snapplanet.io/search
        products:
            S2_MSI_L1C:
                productType: S2
            GENERIC_PRODUCT_TYPE:
                productType: '{productType}'
        download:
            type: AwsDownload
            base_uri: https://tamn.snapplanet.io
            flatten_top_dirs: True
        auth:
            type: AwsAuth
            credentials:
                aws_access_key_id: PLEASE_CHANGE_ME
                aws_secret_access_key: PLEASE_CHANGE_ME
""")
dag.set_preferred_provider("tamn")

2021-03-05 14:27:52,083-15s eodag.config                     [INFO    ] tamn: unknown provider found in user conf, trying to use provided configuration


Search some S2_MSI_L1C products over Luxembourg:

In [4]:
prods_S2L1C, _ = dag.search(productType="S2_MSI_L1C", country="LUX", start="2020-05-01", end="2020-05-15", items_per_page=50)

2021-03-05 14:27:52,263-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: tamn
2021-03-05 14:27:52,264-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://tamn.snapplanet.io/search?datetime=2020-05-01T00:00:00.000Z/2020-05-15T00:00:00.000Z&bbox=5.674051954784829,49.44266714130711,6.242751092156993,50.128051662794235&collections=S2&limit=1&page=1
2021-03-05 14:27:52,473-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://tamn.snapplanet.io/search?datetime=2020-05-01T00:00:00.000Z/2020-05-15T00:00:00.000Z&bbox=5.674051954784829,49.44266714130711,6.242751092156993,50.128051662794235&collections=S2&limit=50&page=1
2021-03-05 14:27:53,003-15s eodag.core                       [INFO    ] Found 35 result(s) on provider 'tamn'


Filter over any item property using crunches:

In [5]:
from eodag.plugins.crunch.filter_property import FilterProperty

prods_S2L1C_filtered = prods_S2L1C.crunch(FilterProperty({"cloudCover": 10, "operator": "lt"}))
len(prods_S2L1C_filtered)

2021-03-05 14:27:53,006-15s eodag.plugins.crunch.filter_property [INFO    ] Finished filtering products. 11 resulting products


11

List available assets from the first retrieved product

In [6]:
[(key, asset["href"]) for key, asset in prods_S2L1C[0].assets.items()]

[('thumbnail',
  'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/preview.jpg'),
 ('metadata',
  'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/metadata.xml'),
 ('tileInfo',
  'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/tileInfo.json'),
 ('productInfo',
  'https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/productInfo.json'),
 ('B1', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B1.jp2'),
 ('B2', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B2.jp2'),
 ('B3', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B3.jp2'),
 ('B4', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B4.jp2'),
 ('B5', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B5.jp2'),
 ('B6', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B6.jp2'),
 ('B7', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B7.jp2'),
 ('B8', 's3://sentinel-s2-l1c/tiles/31/U/GQ/2020/5/14/0/B8.jp2'),
 ('B8A', 's3://sentinel-s2-l1c/tiles/31/U/GQ

Same thing with an unconfigured product type (should match available collections). 

For Tamn Landsat-8 products are available in L8 Collection. Let's search them over Spain:

In [7]:
prods_L8, _ = dag.search(productType="L8", country="ESP", start="2020-05-01", end="2020-05-15")

2021-03-05 14:27:53,039-15s eodag.plugins.manager            [INFO    ] UnsupportedProductType: L8, using generic settings
2021-03-05 14:27:53,039-15s eodag.core                       [INFO    ] Searching product type 'L8' on provider: tamn
2021-03-05 14:27:53,040-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://tamn.snapplanet.io/search?datetime=2020-05-01T00:00:00.000Z/2020-05-15T00:00:00.000Z&bbox=-9.392883673530648,35.946850083961465,3.0394840836805486,43.74833771420099&collections=L8&limit=1&page=1
2021-03-05 14:27:53,292-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://tamn.snapplanet.io/search?datetime=2020-05-01T00:00:00.000Z/2020-05-15T00:00:00.000Z&bbox=-9.392883673530648,35.946850083961465,3.0394840836805486,43.74833771420099&collections=L8&limit=20&page=1
2021-03-05 14:27:53,773-15s eodag.core                       [INFO    ] Found 98 result(s) on provider 'tamn'


In [8]:
[(key, asset["href"]) for key, asset in prods_L8[0].assets.items()]

[('thumbnail',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_thumb_large.jpg'),
 ('metadata',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_MTL.txt'),
 ('B1',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_B1.TIF'),
 ('B2',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_B2.TIF'),
 ('B3',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_B3.TIF'),
 ('B4',
  'https://landsat-pds.s3.amazonaws.com/c1/L8/196/035/LC08_L1TP_196035_20200514_20200527_01_T1/LC08_L1TP_196035_20200514_20200527_01_T1_B4.TIF'),
 ('B5',
  'https://landsat-pds.s3.amazonaws.com/c1/L8

## STAC Static catalog

EODAG can load items from a STAC static catalog. This can be useful for filtering and downloading them.

Here is an example with a catalog from https://stacindex.org/catalogs/spot-orthoimages-canada-2005.

When using `load_stac_items()` we must provide an already configured STAC provider, as its configuration will be used to map item metadata to `EOProduct`. The provider's download configuration will also be used for downloading the static catalog item assets.

*Please note that STAC static catalog search will be implemented as a new search plugin in a future EODAG release to permit using `eodag.search()` with easier configuration*

Let's add a new provider adapted to new provider for this specific catalog, using `StacSearch` and `HTTPDownload` (no authentication needed for download):

In [9]:
# create a workspace
workspace = 'eodag_workspace_stac_client'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

# add the provider
dag.update_providers_config("""
stac_http_provider:
    search:
        type: StacSearch
        api_endpoint: https://fake-endpoint
    products:
        GENERIC_PRODUCT_TYPE:
            productType: '{productType}'
    download:
        type: HTTPDownload
        base_uri: https://fake-endpoint
        flatten_top_dirs: True
        outputs_prefix: %s
""" % os.path.abspath(workspace))

2021-03-05 14:27:53,789-15s eodag.config                     [INFO    ] stac_http_provider: unknown provider found in user conf, trying to use provided configuration


In [10]:
items = dag.load_stac_items(
    "https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/catalog.json", 
    recursive=True, 
    provider="stac_http_provider", # STAC provider adapted to the download configuration
    geometry="POINT EMPTY" # empty geometry as search param that will be ignored when crunching
)

2021-03-05 14:27:58,074-15s eodag.core                       [INFO    ] No product type could be guessed with provided arguments
2021-03-05 14:27:58,075-15s eodag.core                       [INFO    ] Searching product type 'None' on provider: stac_http_provider
2021-03-05 14:28:00,918-15s eodag.core                       [INFO    ] Found 631 result(s) on provider 'stac_http_provider'


In [11]:
len(items)

631

We can filter found products using crunches:

In [12]:
from shapely.geometry import Polygon
from eodag.plugins.crunch.filter_date import FilterDate
from eodag.plugins.crunch.filter_overlap import FilterOverlap

# Filter by date
filtered_by_date = items.crunch(
    FilterDate({"start": "2007-05-01", "end": "2007-05-06"})
)
print("Filtered by date: ", len(filtered_by_date))

# Filter by geometry
search_polygon = Polygon([(-70, 45), (-75, 47), (-80, 47), (-80, 44)])
filtered_by_geom = filtered_by_date.crunch(
    FilterOverlap({"within": True}), 
    geometry=search_polygon
)
print("Filtered by date+geom: ", len(filtered_by_geom))

2021-03-05 14:28:00,967-15s eodag.plugins.crunch.filter_date [INFO    ] Finished filtering products. 21 resulting products
2021-03-05 14:28:00,969-15s eodag.plugins.crunch.filter_overlap [INFO    ] Finished filtering products. 3 resulting products


Filtered by date:  21
Filtered by date+geom:  3


In [13]:
# plot on map
import ipyleaflet as ipyl

m = ipyl.Map(center=(50, -100), zoom=3)

polygon_layer = ipyl.GeoJSON(data=search_polygon.__geo_interface__, style=dict(color='blue'))
m.add_layer(polygon_layer)

filtered_by_date_layer = ipyl.GeoJSON(data=filtered_by_date.as_geojson_object(), style=dict(color='red'))
m.add_layer(filtered_by_date_layer)

filtered_by_geom_layer = ipyl.GeoJSON(data=filtered_by_geom.as_geojson_object(), style=dict(color='green'))
m.add_layer(filtered_by_geom_layer)
m

Map(center=[50, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Download an item from the filtered search results (please note that the thumbnail of the product is not downloadable, and raises a warning):

In [14]:
from eodag.utils import NotebookProgressCallback

path = dag.download(filtered_by_geom[0], progress_callback=NotebookProgressCallback())
path

2021-03-05 14:28:01,065-15s eodag.plugins.download.base      [INFO    ] Download url: https://fake-endpoint/collections/canada_spot5_orthoimages/items/S5_07702_4605_20070505


S5_07702_4605_20070505:   0%|          | 0.00/77.7M [00:00<?, ?B/s]

2021-03-05 14:28:06,839-15s eodag.plugins.download.http      [WARNING ] Unexpected error: 403 Client Error: Forbidden for url: https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07702_4605_20070505/s5_07702_4605_20070505_tn.jpg
2021-03-05 14:28:06,841-15s eodag.plugins.download.http      [WARNING ] Skipping https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/S5_07702_4605_20070505/s5_07702_4605_20070505_tn.jpg
2021-03-05 14:28:26,347-15s eodag.api.product                [INFO    ] Remote location of the product is still available through its 'remote_location' property: https://fake-endpoint/collections/canada_spot5_orthoimages/items/S5_07702_4605_20070505


'file:///home/sylvain/workspace/eodag/examples/eodag_workspace_stac_client/S5_07702_4605_20070505'

In [15]:
!ls {path.replace("file://","")}

S5_07702_4605_20070505_m20_1_lcc00_cog.tif
S5_07702_4605_20070505_m20_2_lcc00_cog.tif
S5_07702_4605_20070505_m20_3_lcc00_cog.tif
S5_07702_4605_20070505_m20_4_lcc00_cog.tif
S5_07702_4605_20070505_p10_1_lcc00_cog.tif
